In [1]:
from sqlalchemy import create_engine, MetaData, Table
import pandas as pd
import uuid
import json

# Replace these with your actual PostgreSQL credentials
user = "hrm"
password = "hrm"
host = "35.223.205.197"  # IP address or hostname
port = "5432"  # Default PostgreSQL port
database = "attendance_tracking"

# Database connection URI for PostgreSQL
pg_database_uri = f'postgresql://{user}:{password}@{host}:{port}/{database}'
pg_engine = create_engine(pg_database_uri)

# SQLite database URI
sqlite_database_uri = 'sqlite:///local_attendance_tracking.db'
sqlite_engine = create_engine(sqlite_database_uri)

# Create MetaData instance for PostgreSQL
pg_metadata = MetaData()
pg_metadata.reflect(bind=pg_engine)

# Specify the tables you want to copy
tables_to_copy = ['tracking_history','tracking_history_error','user_images', 'user_details', 'departments', 'workshops']

# Iterate over specified tables and copy schema and data to SQLite
for table_name in tables_to_copy:
    if table_name in pg_metadata.tables:
        print(f"Copying table: {table_name}")
        pg_table = pg_metadata.tables[table_name]

        # Fetch data from PostgreSQL table
        data = pd.read_sql_table(table_name, pg_engine)

        # Check if the DataFrame is not empty before processing
        if not data.empty:
            # Convert UUID columns to strings and handle complex data
            for col in data.columns:
                if data[col].dtype == 'object':
                    # Convert UUIDs to strings
                    if len(data[col]) > 0 and isinstance(data[col].iloc[0], uuid.UUID):
                        data[col] = data[col].astype(str)

                    # Convert dictionaries or JSON-like objects to strings
                    elif isinstance(data[col].iloc[0], dict):
                        data[col] = data[col].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)

        # Write data to SQLite table
        data.to_sql(table_name, sqlite_engine, if_exists='replace', index=False)

print("Database copy complete.")

Copying table: tracking_history
Copying table: tracking_history_error
Copying table: user_images
Copying table: user_details
Copying table: departments
Copying table: workshops
Database copy complete.
